In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical

In [2]:
NAME = 'EfficientNetB0_good_0812_1350.17-0.45.hdf5'
# /work/power703/cgh/weight/EfficientNetB0_good_0812_1350.17-0.45.hdf5
LOAD_PATH='/work/power703/cgh/weight/'+NAME
OUT_PATH='/work/power703/cgh/weight/'+NAME+'.tflite'
TEST_PATH = '/work/power703/cgh/data_work/test0713/test/'
labels = [
#     "1",
    "2",
    "3",
    "4",
    "5",
#     "6",
#     "7",
    "8",
#     "9",
#     "10",
#     "11",
    "12",
    "13",
    "14",
#     "15",
    # "16"
]
classes = len(labels)

In [3]:
model = keras.models.load_model(LOAD_PATH)
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 257, 500, 1) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 257, 500, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 257, 500, 1)  3           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 259, 501, 1)  0           normalization[0][0]              
_____________________________________________________________________________________

In [4]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

ConverterError: See console for info.
2020-08-12 14:08:13.230500: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2020-08-12 14:08:14.242184: W tensorflow/compiler/mlir/lite/python/graphdef_to_tfl_flatbuffer.cc:144] Ignored output_format.
2020-08-12 14:08:14.242221: W tensorflow/compiler/mlir/lite/python/graphdef_to_tfl_flatbuffer.cc:147] Ignored drop_control_dependency.
2020-08-12 14:08:14.337352: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3000000000 Hz
2020-08-12 14:08:14.340532: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb13c000b20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-08-12 14:08:14.340557: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-08-12 14:08:14.342526: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynami

In [ ]:
# Save the TF Lite model.
with tf.io.gfile.GFile(OUT_PATH, 'wb') as f:
  f.write(tflite_model)

In [ ]:
def get_data_set(PATH):
    X = np.load(PATH + labels[0] + '.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(PATH + label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i + 1)))
    assert X.shape[0] == len(y)
    return X, y

In [ ]:
x_test, y_test = get_data_set(TEST_PATH)
x_test = x_test.reshape(x_test.shape[0], 257, 500, 1)
y_test = to_categorical(y_test, num_classes=classes)

In [ ]:
tmp = np.zeros([1, 257, 500, 1])
ans = np.zeros([x_test.shape[0], classes])
tmp[0] = x_test[0]
input_data = np.array(tmp, dtype=np.float32)

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=OUT_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
for i in range(len(x_test)):
    tmp[0] = x_test[i]
    input_data = np.array(tmp, dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    ans[i] = output_data
    print('.', end='')

In [ ]:
predict_label = np.argmax(ans, axis=1)
y_true = []
for i in range(len(y_test)):
    y_true.append(int(np.argmax(y_test[i])))  

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.3f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure(figsize=[15, 10])
cnf_matrix = confusion_matrix(y_true, predict_label)
plot_confusion_matrix(cnf_matrix, classes=labels, normalize=True,
                      title=NAME)

plt.savefig(NAME+'.png', bbox_inches='tight')

print(classification_report(y_true, predict_label, target_names=labels, digits=3))
print('NAME: ', NAME)